In [1]:
import tensorflow as tf
import tflearn
import numpy as np
import gensim

In [2]:
class LSTM:
    def __init__(self, vocabulary_size=200, num_nodes=128, batch_size=20, num_unrollings=2, name=""):
        '''
        the LSTM process
        '''
        self.num_nodes = num_nodes
        self.reuse = False
        self.vocabulary_size = vocabulary_size
        self.num_nodes = num_nodes
        self.num_unrollings = num_unrollings
        self.batch_size = batch_size
        self.name = name
        
        with tf.variable_scope("{}_LSTM_var".format(self.name) ,reuse=self.reuse):
            with tf.variable_scope("input_gate"):
                self.ix = tf.get_variable("ix", [vocabulary_size, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.im = tf.get_variable("im", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.ib = tf.get_variable("ib", [1, num_nodes], 
                                      tf.float32, tf.constant_initializer(0.1))
            with tf.variable_scope("forget_gate"):
                self.fx = tf.get_variable("fx", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.fm = tf.get_variable("fm", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.fb = tf.get_variable("fb", [1, num_nodes], 
                                     tf.float32, tf.constant_initializer(0.1))
            
            with tf.variable_scope("output_gate"):
                self.ox = tf.get_variable("ox", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.om = tf.get_variable("om", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.ob = tf.get_variable("ob", [1, num_nodes], 
                                     tf.float32, tf.constant_initializer(0.1))
            
            with tf.variable_scope("memory_cell"):
                self.cx = tf.get_variable("cx", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.cm = tf.get_variable("cm", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.cb = tf.get_variable("cb", [1, num_nodes], 
                                     tf.float32, tf.constant_initializer(0.1))
                
            with tf.variable_scope("output_layer"):
                self.w = tf.get_variable("w", [num_nodes, vocabulary_size], tf.float32,
                                        tf.truncated_normal_initializer(stddev=0.1))
                self.b = tf.get_variable("b", [vocabulary_size], tf.float32,
                                        tf.constant_initializer(0.1))
            
            self.saved_output = tf.get_variable("saved_output", [batch_size, num_nodes], 
                                                tf.float32, tf.truncated_normal_initializer(stddev=0.1), 
                                                trainable=False)
            self.saved_state = tf.get_variable("saved_state", [batch_size, num_nodes], 
                                                       tf.float32, tf.truncated_normal_initializer(stddev=0.1), 
                                                       trainable=False)
            '''self.saved_state_backward = tf.get_variable("saved_state_backward", [batch_size, num_nodes], 
                                                       tf.float32, tf.truncated_normal_initializer(stddev=0.1), 
                                                       trainable=False, name="backward_state")'''
                      
        
        self.train_data = list()
        for _ in range(self.num_unrollings + 1):
            self.train_data.append(
                tf.placeholder(tf.float32, shape=[self.batch_size, self.vocabulary_size]))
        self.train_inputs = self.train_data[:self.num_unrollings]
        self.train_labels = self.train_data[1:]
        
        self.outputs = list()
        self.output = self.saved_output
        self.state = self.saved_state
        
        for i in self.train_inputs:
            self.output, self.state = self.lstm_cell(i, self.output, self.state)
            self.outputs.append(self.output)
        
        self.variables = tf.get_collection(tf.GraphKeys.VARIABLES)
    
    def lstm_cell(self, inputs, hidden_layer, state): 
        
        with tf.variable_scope("{}_LSTM_cell".format(self.name), reuse=self.reuse):
            input_gate = tf.sigmoid(tf.matmul(inputs, self.ix) + tf.matmul(hidden_layer, self.im) + self.ib,
                                    name="input_gate")
            forget_gate = tf.sigmoid(tf.matmul(inputs, self.fx) + tf.matmul(hidden_layer, self.fm) + self.fb,
                                    name="forget_gate")
            update_gate = tf.tanh(tf.matmul(inputs, self.cx) + tf.matmul(hidden_layer, self.cm) + self.cb,
                            name="update_gate")
            state = tf.add(tf.mul(forget_gate, state), tf.mul(input_gate, update_gate),
                           name="state")
            output_gate = tf.sigmoid(tf.matmul(inputs, self.ox) + tf.matmul(hidden_layer, self.om) + self.ob,
                                     name="output_gate") 
            output = tf.mul(output_gate, tf.tanh(state), name="output")
        
        #Note that, the "output" is equal to the hidden state
        return output, state   
    
    def lstm_process(self, inputs, hidden_layer, state):
        for i in train_inputs:
            self.output, self.state = self.lstm_cell(i, self.output, self.state)
            self.outputs.append(output)

        return outputs
        '''with tf.control_dependencies([self.saved_output.assign(output),
                                     self.saved_state.assign(state)]):
            
            logits = tf.matmul(tf.concat(0, outputs), self.w) + self.b
            loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(tf.concat(0, logits), 
                                                        tf.concat(0, train_labels), name="loss"))
            
            global_step = tf.Variable(0)
            learning_rate = tf.train.exponential_decay(
                10.0, global_step, 5000, 0.1, staircase=True)
            optimizer = tf.train.RMSPropOptimizer(learning_rate)
            gradient, v = zip(*optimizer.compute_gradients(loss))
            
            
            tf.add_to_collection("loss", loss)
            tf.add_to_collection("logits", logits)
            tf.add_to_collection("train_labels", train_labels)
            
        '''
        #It seems we do not need those lines above
        
    def train(self):
        pass
    
    def __call__(self, inputs):
        self.lstm_process(inputs)
        

In [3]:
class attention:
    def __init__(self, h, s, name=""):
        '''
        Caculate the attention weight.
        TODO:
            I think it seems appropriate to transform the class 
            into a function...
        arguments:
            h: all the encoder's hidden states, 
            each of them has a size of [2*Hidden_size, Batch_size]
            s: [2*Hidden_size, Batch_size], decoder's hidden state
        '''
        self.name = name
        self.attn_logits = list()
        self.a = list()
        
        #some variables
        with tf.variable_scope("{}_attention_model".format(self.name)):
            self.Ua = tf.get_variable("Ua", [int(h[0].get_shape()[1])/2, int(h[0].get_shape()[1])],
                                     tf.float32, tf.truncated_normal_initializer(stddev=0.1))
            self.Wa = tf.get_variable("Wa", [int(s.get_shape()[1]), int(s.get_shape()[1])],
                                     tf.float32, tf.truncated_normal_initializer(stddev=0.1))
            #the following bias does not showed in original paper
            self.Ba = tf.get_variable("Ba", [1, int(s.get_shape()[1])], 
                                      tf.float32,
                                      tf.constant_initializer(0.1))
            self.Va = tf.get_variable("Va", [1, int(s.get_shape()[1])], 
                                      tf.float32,
                                      tf.constant_initializer(0.1))
            
            #caculate the attention weight
            for h_j in h:
                self.attn_mlp = tf.tanh(
                    tf.matmul(self.Wa, tf.transpose(s)) + tf.matmul(self.Ua, tf.transpose(h_j)))
                self.attn_logits.append(tf.reshape(
                        tf.matmul(self.Va, self.attn_mlp), shape=[1])[0])
                
            self.a = tf.reshape(tf.nn.softmax(self.attn_logits),
                                shape=[len(h)], name="attention_value")
        
    def get_attention():
        return self.a

In [4]:
class Discriminator():
    def __init__(self, s, conv_sizes, num_filters, dropout_keep_prob,
                name=""):
        '''
        The Discriminator, uses Convolutional Neural Network to 
        classify the input sentence. See 
        http://www.people.fas.harvard.edu/~yoonkim/data/emnlp_2014.pdf
        for detail. Those code below with some inspiration from
        [https://github.com/dennybritz/cnn-text-classification-tf/blob/master/text_cnn.py]
        
        Arguments:
        s: the sentence representation caculated by LSTM process, 
        which has a size of [batch_size, hidden_layer_size]
        '''
        
        self.dropout_keep_prob = dropout_keep_prob
        self.num_filters = num_filters
        self.conv_sizes = conv_sizes
        self.hidden_size = int(s.get_shape()[2])
        print(self.hidden_size)
        self.pooled_outputs = list()
        for i, conv_size in enumerate(conv_sizes):
            with tf.variable_scope("conv-maxpooling-{}"
                                  .format(conv_size)):
                #filter_shape = [conv_size, self.hidden_size, 1, num_filters]
                W = tf.get_variable("W", [1, conv_size, 1, num_filters],
                                   tf.float32,
                                   tf.truncated_normal_initializer(stddev=0.1))
                print(W.get_shape())
                b = tf.get_variable("b", [num_filters], tf.float32,
                                   tf.constant_initializer(0.1))
                
                conv = tf.nn.conv2d(s, W, [1,1,1,1], 'VALID', name="conv")
                print(conv.get_shape())
                h = tf.nn.relu(conv + b)
                
                pooled = tf.nn.max_pool(h,
                                       ksize=[1, 1, self.hidden_size-conv_size+1, 1],
                                       strides=[1,1,1,1],
                                       padding='VALID',
                                       name="max_pooling")
                print(pooled.get_shape())
                self.pooled_outputs.append(pooled)
                
    
    
        
        

In [5]:
L = LSTM(batch_size=1, num_unrollings=2, name="TEST")

In [6]:
word_model = gensim.models.Word2Vec.load("models/word2vec.model")

In [7]:
test_word1 = word_model["china"]
test_word2 = word_model["uk"]
test_word3 = word_model["america"]

In [8]:
test_sentence = [test_word1, test_word2, test_word3]

In [9]:
h = tf.Variable(tf.truncated_normal((1,128)))
state = tf.Variable(tf.truncated_normal((1,128)))
test_inputs = tf.placeholder(tf.float32, shape=[1, 200])

In [10]:
h_j = list()
for _ in range(10):
    h_j.append(tf.Variable(tf.truncated_normal((1, 256))))
s_j = tf.Variable(tf.truncated_normal((1, 128)))

In [11]:
A = attention(h_j, s_j, "1224")

In [12]:
s_j = tf.reshape(s_j, [1, 1, 128, 1])

In [13]:
s_j.get_shape()[2]

Dimension(128)

In [14]:
D = Discriminator(s_j, [3, 4, 5, 6], 8, 0.5, "dd")

128
(1, 3, 1, 8)
(1, 1, 126, 8)
(1, 1, 1, 8)
(1, 4, 1, 8)
(1, 1, 125, 8)
(1, 1, 1, 8)
(1, 5, 1, 8)
(1, 1, 124, 8)
(1, 1, 1, 8)
(1, 6, 1, 8)
(1, 1, 123, 8)
(1, 1, 1, 8)


In [15]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    feed_dict = dict()
    for i in range(3):
        feed_dict[L.train_data[i]] = np.reshape(test_sentence[i],(1,200))
    out = sess.run(L.outputs, feed_dict=feed_dict)
    print(out)

[array([[-0.00964425, -0.23564938, -0.56542951,  0.11543711,  0.03337646,
         0.00332987, -0.08882207,  0.01510722,  0.13301107, -0.16482908,
        -0.02039593,  0.03089569,  0.13372445,  0.05805426, -0.00515711,
         0.01545899, -0.09739969,  0.10948285, -0.27909574, -0.16596355,
         0.09395462, -0.04627905,  0.12152587,  0.2531153 ,  0.61662143,
        -0.09192392,  0.23146346,  0.01679718, -0.0521368 ,  0.01266803,
         0.12985617,  0.40175137, -0.13689633,  0.13738512,  0.05932658,
         0.09846523, -0.07685275, -0.05234725, -0.38297564, -0.17925805,
        -0.17254889, -0.02166946, -0.06030574,  0.18364599,  0.07565676,
         0.26444671,  0.03029758,  0.5055086 , -0.25924441, -0.02806696,
         0.18840306, -0.06652591,  0.56122327,  0.02877892, -0.15657791,
        -0.15958373, -0.05559675,  0.05910024, -0.26470396,  0.24412498,
        -0.08462624, -0.00373746,  0.0043321 ,  0.17686912,  0.18839963,
         0.15735877,  0.11163829, -0.20740998, -0.

In [16]:
out[0].shape

(1, 128)

In [17]:
ll = [5,4,3,2,1]
for i, l in enumerate(ll):
    print(i,l)

0 5
1 4
2 3
3 2
4 1
